In [1]:
START_YEAR = 2005
SPORT_IDS = [1,11,12,13,14,15,16]
MEXICAN_LEAGUE_ID = 125 # For some years, mlb returns it for AAA level, which I do not want
DSL_LEAGUE_ID = 130 # Set this to level 17

In [2]:
CURRENT_SEASON = 2024
ONLY_UPDATE_CURRENT_SEASON = False

In [3]:
STOP_MONTH = 8

In [2]:
import requests
import sqlite3

In [5]:
from tqdm import tqdm
import threading

In [6]:
import traceback

Generate Game Log Data

In [7]:
dbWriteLock = threading.Lock()

In [8]:
def GeneratePitcherYearGameLogs(db, mlbId, year, startMonth=0, endMonth=13):
    cursor = db.cursor()
    # Check if data already exists
    if cursor.execute(f"SELECT COUNT(*) FROM Player_Pitcher_GameLog WHERE mlbId='{mlbId}' AND Year='{year}' AND Month >'{startMonth}'").fetchone()[0] > 0:
        return
    
    gameLogs = []
    response = requests.get(f"https://statsapi.mlb.com/api/v1/people/{mlbId}/stats?stats=gameLog&leagueListId=mlb_milb&group=pitching&gameType=R&sitCodes=1,2,3,4,5,6,7,8,9,10,11,12&hydrate=team&language=en&season={year}")
    if response.status_code != 200:
        print(f"Status code {response.status_code} for id={mlbId} year={year}")
        return
    try:
        # Make sure any data exists
        try:
            games = response.json()["stats"][0]["splits"]
        except:
            return
        
        # Add each game
        for game in games:
            if game["team"]["league"]["id"] == MEXICAN_LEAGUE_ID:
                continue
            
            _, month, day = game["date"].split("-")
            if int(month) < int(startMonth) or int(month) >= int(endMonth):
                continue
            
            gameId = int(game["game"]["gamePk"])
            isHomeGame = int(game["isHome"])
            if isHomeGame:
                homeTeamId = int(game["team"]["id"])
            else:
                homeTeamId = int(game["opponent"]["id"])
            level = int(game["sport"]["id"])
            stats = game["stat"]
            bf = int(stats["battersFaced"])
            outs = int(stats["outs"])
            groundOuts = int(stats["groundOuts"])
            airOuts = int(stats["airOuts"])
            r = int(stats["runs"])
            er = int(stats["earnedRuns"])
            h = int(stats["hits"])
            k = int(stats["strikeOuts"])
            bb = int(stats["baseOnBalls"])
            hbp = int(stats["hitByPitch"])
            double = int(stats["doubles"])
            triples = int(stats["triples"])
            hr = int(stats["homeRuns"])
            teamId = game["team"]["id"]
            leagueId = game["league"]["id"]
            gameLogs.append((gameId, int(mlbId), int(day), int(month), int(year), bf, outs, groundOuts, airOuts, r, er, h, k, bb, hbp, double, triples, hr, level, homeTeamId, teamId, leagueId))
            
        # Write to database
        with dbWriteLock:
            cursor.execute("BEGIN TRANSACTION")
            cursor.executemany("INSERT INTO Player_Pitcher_GameLog('gameId', 'mlbId', 'Day', 'Month','Year','battersFaced','outs','go','ao','r','er','h','k','bb','hbp','2B','3B','HR','Level','HomeTeamId', 'TeamId', 'LeagueId') VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", gameLogs)
            cursor.execute("END TRANSACTION")
            db.commit()
            
    except Exception as e:
        print(f"Exception On Id={mlbId} year={year}: {e}")
        traceback.print_exc()
        return

In [9]:
# Create dictionary of player ids-year combination
# Don't need any data, just a lookup table
playerYearDict = {}
for year in tqdm(range(START_YEAR, CURRENT_SEASON + 1)):
    if ONLY_UPDATE_CURRENT_SEASON and year != CURRENT_SEASON:
            continue

    for sportId in SPORT_IDS:
        #print(f"Getting players for Year={year} SportId={sportId}")
        response = requests.get(f"https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?stitch_env=prod&season={year}&sportId={sportId}&stats=season&group=pitching&gameType=R&limit=5000&offset=0&sortStat=homeRuns&order=desc")
        if response.status_code != 200:
            print(f"Code {response.status_code} for Year={year} and sportId={sportId}")
            continue

        responseJson = response.json()
        jsonPlayers = responseJson['stats']
        for player in jsonPlayers:
            playerString = str(player["playerId"]) + "," + str(year)
            playerYearDict[playerString] = None

100%|██████████| 20/20 [04:17<00:00, 12.87s/it]


In [10]:
NUM_THREADS = 16

In [11]:
def PitcherGameLogThreadFunction(data, threadIdx):
    db = sqlite3.connect("BaseballStats.db")
    cursor = db.cursor()
    with dbWriteLock:
        cursor.execute("PRAGMA journal_mode = WAL")
        db.commit()
    cursor = db.cursor()
    global threadCompleteCounts
    for d in data:
        try:
            mlbId, year = d.split(",")
            if year == CURRENT_SEASON:
                months = cursor.execute(f"SELECT DISTINCT Month FROM Player_Pitcher_GameLog WHERE mlbId='{mlbId}' AND Year='{year}' ORDER BY Month DESC").fetchall()
                if len(months) == 0:
                    startMonth = 0
                else:
                    startMonth = months[0][0]
                GeneratePitcherYearGameLogs(db, mlbId, year, startMonth, STOP_MONTH)
            else:
                if cursor.execute(f"SELECT COUNT(*) FROM Player_Pitcher_GameLog WHERE mlbId='{mlbId}' AND Year='{year}'").fetchone()[0] == 0:
                    GeneratePitcherYearGameLogs(db, mlbId, year)
        except Exception as e:
            print(f"Error for id={mlbId} year={year}: {e}")
        finally:
            threadCompleteCounts[threadIdx] += 1
    
        

In [12]:
db = sqlite3.connect("BaseballStats.db")
db.rollback()

playerYears = list(playerYearDict.keys())

threads = []
threadCompleteCounts = [0] * NUM_THREADS
for i in range(NUM_THREADS):
    thread = threading.Thread(target=PitcherGameLogThreadFunction, args=[playerYears[len(playerYears) * i // NUM_THREADS : len(playerYears) * (i + 1) // NUM_THREADS], i])
    threads.append(thread)
    thread.start()
    
progressBar = tqdm(total=len(playerYears))

# Start progress bar
keepTimerRunning = True
def UpdateTimer():
    if keepTimerRunning:
        threading.Timer(5.0, UpdateTimer).start()
    
    count = 0
    global threadCompleteCounts
    for i in range(NUM_THREADS):
        count += threadCompleteCounts[i]
    
    global progressBar
    progressBar.n = count
    progressBar.last_print_n = progressBar.n
    progressBar.refresh()
    
UpdateTimer()

for thread in threads:
    thread.join()
    
keepTimerRunning = False

 13%|█▎        | 11761/91948 [11:21<1:17:23, 17.27it/s]

Error for id=598465 year=2017: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error for id=622380 year=2012: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


100%|█████████▉| 91942/91948 [1:28:38<00:00, 17.29it/s]

KeyboardInterrupt: 

In [3]:
db = sqlite3.connect("BaseballStats.db")
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

cursor.execute("UPDATE Player_Pitcher_GameLog SET Level=? WHERE LeagueId=?", (17, DSL_LEAGUE_ID))

cursor.execute("END TRANSACTION")
db.commit()

In [13]:
# PitcherGameLogThreadFunction([
#     "598465,2017",
#     "622380,2012"
# ], 0)

91950it [2:54:46,  8.77it/s]                           